In [1]:
import pandas as pd
import json
import pickle
import codecs

from google.cloud import storage

In [2]:
client = storage.Client()
bucket = client.get_bucket("rjr-portal-da-transparencia")

In [36]:
def save_maps(maps):
    with open('maps.json', 'w', encoding='utf-8') as file:
        json.dump(maps, file, ensure_ascii=False)

    blob = bucket.blob(f"aux_tables/maps.json")
    blob = blob.upload_from_filename("maps.json")

In [41]:
df = pd.read_csv(f"gs://rjr-portal-da-transparencia/aux_tables/ufs.csv", 
                 engine="python", sep=',', quotechar='"', header=0, encoding="utf8")

maps = {}

In [42]:
# tables = ["CO_CURSO_EDUC_PROFISSIONAL", "CO_ORGAO_REGIONAL"]
# for table in tables:
#     df_ = pd.read_csv(f"gs://rjr-portal-da-transparencia/aux_tables/{table}.csv", 
#                  engine="python", sep=',', quotechar='"', header=0, usecols=[0,1], encoding="utf8")
#     maps[table] = dict(df_.values)

In [43]:
names = ["CO_REGIAO", "CO_UF", "CO_MESORREGIAO", 
        "CO_MICRORREGIAO", "CO_MUNICIPIO"]
cols = [[0, 1], [2, 3], [2, 5, 6], [2, 7, 8], [9, 10]]
names_cols = zip(filenames, cols)


for name, cols_ in names_cols:
    
    df_ = df.iloc[:, cols_].drop_duplicates()
    if len(cols_) == 2:
        map_ = dict(df_.values)
    elif name == "CO_MESORREGIAO":
        map_ = {int(f"{col1}{col2:02}"): col3
                       for col1, col2, col3 
                       in df_.values} 
    else:
        map_ = {int(f"{col1}{col2:03}"): col3
                       for col1, col2, col3 
                       in df_.values} 
        
    
    maps[name] = map_    
    #save_dict(df_dict, filename)

In [44]:
maps["TP_SITUACAO_FUNCIONAMENTO"] = {
    1: "Em Atividade",
    2: "Paralisada",
    3: "Extinta (ano do censo)",
    4: "Extinta em anos anteriores"
}

maps["TP_DEPENDENCIA"] = {
    1: "Federal",
    2: "Estadual",
    3: "Municipal",
    4: "Privada"
}


maps["TP_LOCALIZACAO"] = {
    1: "Urbana",
    2: "Rural"
}

maps["TP_LOCALIZACAO_DIFERENCIADA"] = {
    0: "A escola não está em área de localização diferenciada",
    1: "Área de assentamento",
    2: "Terra indígena",
    3: "Área onde se localiza comunidade remanescente de quilombos"
}

maps["TP_CATEGORIA_ESCOLA_PRIVADA"] = {
    1: "Particular",
    2: "Comunitária",
    3: "Confessional",
    4: "Filantrópica"
}

maps["TP_CONVENIO_PODER_PUBLICO"] = {
    1: "Municipal",
    2: "Estadual",
    3: "Estadual e Municipal",
}

maps["TP_REGULAMENTACAO"] = {
    0: "Não",
    1: "Sim", 
    2: "Em tramitação"
}


maps["TP_RESPONSAVEL_REGULAMENTACAO"] = {
    1: "Federal",
    2: "Estadual",
    3: "Municipal",
    4: "Estadual e Municipal",
    5: "Federal e Estadual",
    6: "Federal, Estadual e Municipal"
}

maps["TP_OCUPACAO_PREDIO_ESCOLAR"] = {
    1: "Próprio",
    2: "Alugado",
    3: "Cedido"
}

maps["TP_REDE_LOCAL"] = {
    0: "Não há rede local interligando computadores",
    1: "A cabo",
    2: "Wireless",
    3: "A cabo e Wireless"
}


maps["TP_INDIGENA_LINGUA"] = {
    1: "Somente em Língua Indígena",
    2: "Somente em Língua Portuguesa",
    3: "Em Língua Indígena e em Língua Portuguesa"
}

maps["TP_PROPOSTA_PEDAGOGICA"] = {
    0: "Não",
    1: "Sim",
    2: "A escola não possui projeto político pedagógico/proposta pedagógica"
}

maps["TP_AEE"] = {
    0: "Não oferece",
    1: "Não exclusivamente",
    2: "Exclusivamente"
}

maps["TP_ATIVIDADE_COMPLEMENTAR"] = {
    0: "Não oferece",
    1: "Não exclusivamente",
    2: "Exclusivamente"
}

maps["TP_OCUPACAO_GALPAO"] = {
    1: "Próprio",
    2: "Alugado",
    3: "Cedido"
}

In [45]:
save_maps(maps)